In [2]:
import requests
import json
import pandas as pd
import os

In [23]:
ReddingBikes = pd.read_csv('../data/ReddingBikesDF.csv')

In [ ]:
ReddingBikes.head()

,empty_slots,free_bikes,id,latitude,longitude,name,timestamp,extra.address,extra.ebikes,extra.has_ebikes,extra.last_updated,extra.normal_bikes,extra.rental_uris.android,extra.rental_uris.ios,extra.renting,extra.returning,extra.uid
0,2,2,bb7665847eda4d3b5654ef23584bba2f,40.58002,-122.38890,Market and Gold Street,2024-05-21T04:46:17.148000Z,Market Street & Gold Street,2,True,1716266776,0,https://www.bcycle.com/applink?system_id=bcycl...,https://www.bcycle.com/applink?system_id=bcycl...,1,1,bcycle_redding_7427
1,3,2,3bb59ffd3799adff3f475e8d3985532f,40.58352,-122.39042,Market Center South,2024-05-21T04:46:17.152000Z,Market Street & Yuba Street,2,True,1716266776,0,https://www.bcycle.com/applink?system_id=bcycl...,https://www.bcycle.com/applink?system_id=bcycl...,1,1,bcycle_redding_7426
2,2,3,08aae6d19bdfffb93561437caf892f36,40.58435,-122.39067,Market Center North,2024-05-21T04:46:17.153000Z,1371 Butte Street,3,True,1716266776,0,https://www.bcycle.com/applink?system_id=bcycl...,https://www.bcycle.com/applink?system_id=bcycl...,1,1,bcycle_redding_7425
3,3,2,dd5c0ef4079f9e920b3cdba5c3ca167c,40.59502,-122.40150,Diestelhorst Bridge,2024-05-21T04:46:17.160000Z,Benton Drive,2,True,1716266776,0,https://www.bcycle.com/applink?system_id=bcycl...,https://www.bcycle.com/applink?system_id=bcycl...,1,1,bcycle_redding_7424
4,3,3,64a7ac84a24f102734b87abc4ba71af0,40.57337,-122.38177,Redding City Hall,2024-05-21T04:46:17.161000Z,777 Cypress Ave,3,True,1716266776,0,https://www.bcycle.com/applink?system_id=bcycl...,https://www.bcycle.com/applink?system_id=bcycl...,1,1,bcycle_redding_7423


# [Foursquare](https://docs.foursquare.com/developer/reference/places-api-overview)

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [78]:
def get_venues_fs(latitude, longitude, radius, api_key, categories, limit):
    """
    Get venues from foursquare with a specified place type and coordinates.
    Args:
        latitude (float): latitude for query (must be combined with longitude)
        longitude (float): longitude for query (must be combined with latitude)
        api_key (str): foursquare API to use for query
        categories (str) : Foursquare-recognized place type. If not passed no place_type will be specified. Separate ids with commas
        limit (int): Number of results to return, max 50

    Returns:
        response: response object from the requests library.
    """
    headers = {
        "accept": "application/json",
        "Authorization": api_key
    }

    ll=str(latitude) + ',' + str(longitude)

    url = f'https://api.foursquare.com/v3/places/search?ll={ll}&radius={radius}&categories={categories}&limit={str(limit)}'

    results = requests.get(url, headers=headers)

    return results

Foursquare [category codes](https://docs.foursquare.com/data-products/docs/categories) for Dining & Drinking and POIs
* 10000	Arts and Entertainment
* 13000   Dining & Drinking
* 16000	Landmarks and Outdoors
* 18000	Sports and Recreation


In [30]:
test['results'][0]['categories'][0]['id']

13383

'10'

In [70]:
FOURSQUARE_KEY = os.getenv('Foursquare_API_KEY')
radius = 1000
categories = '10000,13000,16000,18000'
limit = 50

In [74]:
test = get_venues_fs(49.2827, -123.1207, '5000', FOURSQUARE_KEY, categories, 50)

In [77]:
test

{'message': 'invalid categoryId(s) provided : 18000?limit=50; see https://docs.foursquare.com/docs/categories'}

In [75]:
test = test.json()

In [76]:
len(test['results'])

KeyError: 'results'

In [79]:
POIs = [] # Total POIs
POICategories = []
POINames = []
entertainment = [] # Total Entertainments
foodDrinks = [] # Total Food And Drinks
landmarks = [] # Total landmarks
recreational = [] # Total recreationals

for i in range(len(ReddingBikes)):
    venues = get_venues_fs(ReddingBikes['latitude'][i], ReddingBikes['longitude'][i], radius, FOURSQUARE_KEY, categories, limit).json()['results']
    
    POIs.append(len(venues))

    venueCategories = []
    venueName = []
    for venue in venues:
        venueCategories.append(venue['categories'][0]['id'])
        venueName.append(venue['name'])
    venueCategories.sort()
    venueName.sort()
    POICategories.append(venueCategories)
    POINames.append(venueName)
    


In [80]:
pd.DataFrame(POICategories)

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,10000,10000,10003,10004,10004,10039,10039,10039,10051,10051,...,18000.0,18021.0,18025.0,18025.0,18038.0,18076.0,NaN,NaN,NaN,NaN
1,10000,10000,10000,10003,10004,10004,10039,10039,10039,13000,...,17121.0,18000.0,18021.0,18021.0,18025.0,18025.0,18036.0,18038.0,18076.0,NaN
2,10000,10000,10000,10003,10004,10004,10006,10039,10039,10039,...,17121.0,18000.0,18021.0,18021.0,18025.0,18036.0,18038.0,18076.0,NaN,NaN
3,10000,10000,10000,10004,10039,11020,13031,13039,13055,13064,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10000,10004,10004,10051,10051,10051,13000,13002,13006,13032,...,18021.0,18025.0,18025.0,18025.0,18025.0,18027.0,18028.0,18036.0,18036.0,18038.0
5,10000,10000,10000,10000,10003,10004,10004,10004,10039,10039,...,16032.0,17121.0,18000.0,18021.0,18021.0,18025.0,18036.0,18038.0,18048.0,18076.0
6,10000,10000,10004,10039,10039,10051,10051,10051,13000,13006,...,18021.0,18025.0,18025.0,18025.0,18025.0,18027.0,18028.0,18036.0,NaN,NaN
7,10000,10000,10003,10004,10004,10037,10039,10039,10039,13000,...,18021.0,18025.0,18036.0,18038.0,18076.0,NaN,NaN,NaN,NaN,NaN
8,10000,10000,10000,10003,10004,10004,10004,10039,10039,10039,...,17121.0,18000.0,18021.0,18021.0,18025.0,18036.0,18038.0,18048.0,18076.0,NaN
9,10000,10003,10004,10004,10039,10039,10039,10051,10051,13000,...,18021.0,18021.0,18025.0,18025.0,18025.0,18038.0,18076.0,NaN,NaN,NaN


### Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

### Put your parsed results into a DataFrame

# [Yelp API](https://docs.developer.yelp.com/reference/v3_business_search)

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [3]:
YelpID = os.getenv('YelpClientID')
YelpKey = os.getenv('YelpAPIKey')


In [16]:
import requests

YelpID = os.getenv('YelpClientID')
YelpKey = os.getenv('YelpAPIKey')

url = "https://api.yelp.com/v3/businesses/search?location=Vancouver&radius=10000&categories=&sort_by=best_match&limit=50"

headers = {
    "accept": "application/json",
    "user-agent": "Kenneth",
    "Authorization": f"Bearer {YelpKey}"
}

response = requests.get(url, headers=headers)

print(response)


<Response [200]>


In [15]:
import requests

url = "https://api.yelp.com/v3/businesses/search?sort_by=best_match&limit=20"

headers = {
    "accept": "application/json",
    "Authorization": "NXLSPKb8ETcOmGlEBNtkWE0yeSwOtutGC7SEyhDKNJWYwqO6i6SEilzBLZ-Iq--eITeMHFZoJlINDwSqbwq8ptqrTlgVh7a3rBdSZ73xSYt7529RS-1H_zZMYutLZnYx"
}

response = requests.get(url, headers=headers)

print(response.text)

{"error": {"code": "VALIDATION_ERROR", "description": "Please specify a location or a latitude and longitude"}}


### Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

### Put your parsed results into a DataFrame

# Comparing Results

### Which API provided you with more complete data? Provide an explanation. 

### Get the top 10 restaurants according to their rating